# 단기 기억 문제 해결하기

## 장단기 메모리 (long short-term memory, LSTM)


<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-9.png">

$
\begin{split}
\mathbf{i}_{(t)}&=\sigma({\mathbf{W}_{xi}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hi}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_i)\\
\mathbf{f}_{(t)}&=\sigma({\mathbf{W}_{xf}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hf}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_f)\\
\mathbf{o}_{(t)}&=\sigma({\mathbf{W}_{xo}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{ho}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_o)\\
\mathbf{g}_{(t)}&=\operatorname{tanh}({\mathbf{W}_{xg}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hg}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_g)\\
\mathbf{c}_{(t)}&=\mathbf{f}_{(t)} \otimes \mathbf{c}_{(t-1)} \, + \, \mathbf{i}_{(t)} \otimes \mathbf{g}_{(t)}\\
\mathbf{y}_{(t)}&=\mathbf{h}_{(t)} = \mathbf{o}_{(t)} \otimes \operatorname{tanh}(\mathbf{c}_{(t)})
\end{split}
$


박스 안을 들여다보지 않는다면 LSTM 셀은 상태가 두개의 벡터 $\mathbf{h}_{(t)}$와 $\mathbf{c}_{(t)}$ (c는 셀(cell)을 의미)로 나뉜다는 것을 빼고는 정확히 일반 셀처럼 보입니다. $\mathbf{h}_{(t)}$를 단기 상태(short-term state), $\mathbf{c}_{(t)}$를 장기 상태(long-term state)라고 생각할 수 있습니다. 박스 안을 들여다 보면, 핵심 아이디어는 네트워크가 장기 상태에 저장할 것, 버릴 것, 그리고 읽어들을 것을 학습하는 것입니다. 

장기 기억 $\mathbf{c}_{(t-1)}$은 네트워크를 왼쪽에서 오른쪽으로 관통하면서, 삭제 게이트(forget gate)를 지나 일부 기억을 잃고, 그런 다음 덧셈 연산으로 새로운 기억 일부를 추가합니다(입력 게이트, input gate에서 선택한 기억을 추가합니다). 만들어진 $\mathbf{c}_{(t)}$는 다른 추가 변환 없이 바로 출력으로 보내집니다. 그래서 타임 스텝마다 일부 기억이 삭제되고 일부 기억이 추가됩니다. 또한 덧셈 연산 후 이 장기(long-term) 상태가 복사되 tanh 함수로 전달됩니다. 그런 다음 이결과는 출력 게이트(output gate)에 의해 걸러집니다. 이 단기 상태 $\mathbf{h}_{(t)}$를 만듭니다. 


-  주 층은 $\mathbf{g}_{(t)}$를 출력하는 층입니다. 이 층은 현재 입력 $\mathbf{x}_{(t)}$와 이전의 (단기) 상태 $\mathbf{h}_{(t-1)}$을 분석하는 일반적인 역할을 담당합니다. 기본 셀에서는 이 층외에 다른 것은 없고 바로 $\mathbf{y}_{(t)}$와 $\mathbf{h}_{(t)}$로 출력됩니다. 반대로 LSTM에서는 이 층의 출력이 곧 바로 나가지 않고, 대신 장기 상태에 가장 중요한 부분이 저장됩니다 (나머지는 버림)

- 세 개의 다른 층은 게이트 제어기(gate controller)입니다. 이들은 로지스틱 활성화 함수를 사용하기 때문에 출력의 범위가 0에서 1 사이입니다. 그림에서 보듯이 이들의 출력은 원소별 곱셈연산으로 주입되어, 0을 출력하면 게이트를 닫고 1을 출력하면 게이트를 엽니다.

- 삭제 게이트($\mathbf{f}_{(t)}$가 제어함)는 장기 상태의 어느 부분이 삭제되어야 하는지 제어합니다

- 입력 게이터($\mathbf{i}_{(t)}$가 제어함)는 $\mathbf{g}_{(t)}$의 어느 부분이 장기 상태에 더해져야 하는지 제어합니다.

- 마지막으로 출력 게이트($\mathbf{o}_{(t)}$가 제어함)는 장기 상태의 어느 부분을 읽어서 이 타임 스텝의 $\mathbf{h}_{(t)}$와 $\mathbf{y}_{(t)}$로 출력해야 하는지 제어합니다.


In [1]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
model.evaluate(X_valid, Y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

## 게이트 순환 유닛(gated recurrent unit, GRU) 셀

GRU 셀은 2014년 조경현(https://arxiv.org/abs/1406.1078) 등의 논문에서 제안됐습니다. 이 논문에서는 앞서 언급한 인코더-디코더 네트워크도 소개합니다.

<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-10.png">


GRU 셀은 LSTM 셀의 간소화된 버전이고 유사하게 작동하는 것처럼 보입니다. 간소화된 주요 내용은 다음과 같습니다.

- 두 상태 벡터가 하나의 벡터 $\mathbf{h}_{(t)}$로 합쳐졌습니다.

- 하나의 게이트 제어기 $\mathbf{z}_{(t)}$가 삭제 게이트와 입력 게이트를 모두 제어합니다. 게이트 제어기가 1을 출력하면 사겢 게이트가 열리고(= 1 ) 입력 게이트가 닫힙니다 (1 - 1 = 0). 게이트 제어기가 0을 출력하면 그 반대가 됩니다. 다시 말해 기억이 저장될 떄마다 저장될 위치가 먼저 삭제됩니다. 사실 이것 자체는 흔한 LSTM 셀의 변종입니다.

- 출력 게이트가 없습니다. 즉, 전체 상태 벡터가 매 타임 스텝마다 출력됩니다. 그러나 이전 상태의 어느 부분이 주 층($\mathbf{g}_{(t)}$)에 노출될지 제어하는 새로운 게이트 제어기  $\mathbf{r}_{(t)}$가 있습니다.

GRU 계산식


$
\begin{split}
\mathbf{z}_{(t)}&=\sigma({\mathbf{W}_{xz}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hz}}^T \mathbf{h}_{(t-1)}) \\
\mathbf{r}_{(t)}&=\sigma({\mathbf{W}_{xr}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hr}}^T \mathbf{h}_{(t-1)}) \\
\mathbf{g}_{(t)}&=\operatorname{tanh}\left({\mathbf{W}_{xg}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hg}}^T (\mathbf{r}_{(t)} \otimes \mathbf{h}_{(t-1)})\right) \\
\mathbf{h}_{(t)}&=(1-\mathbf{z}_{(t)}) \otimes \mathbf{h}_{(t-1)} + \mathbf{z}_{(t)} \otimes \mathbf{g}_{(t)}
\end{split}
$


케라스는 keras.layers.GRU층을 제공합니다. 이 층을 사용하려면 SimpleRNN이나 LSTM을 GRU로 바꾸면 됩니다.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
model.evaluate(X_valid, Y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()